# Notebook Description
This notebook is built using Jupyter notebook, running in a docker container from https://github.com/canwaf/lachesis. The sole purpose is to visualise the UK's COVID-19 vaccine rollout.

## License
This notebook is provided under license of the parent repo, the data accessed and visualised is under OLG v3.0. https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter
import matplotlib.patches as mpatches
from matplotlib import gridspec
import numpy as np
import seaborn as sns
import json
import requests

In [2]:
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import ExpiresAfter

# Population Information
First we collect the population tables for the countries of the United Kingdom by age, and shape it accordingly. [UK's population estimates (mid-year)](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/annualmidyearpopulationestimates/mid2019estimates) are from the ONS and licensed under [Open Government License v3.0](https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/).

In [3]:
# Object manages the session
sess = requests.session()
cached_sess = CacheControl(sess, cache=FileCache('.cache'), heuristic=ExpiresAfter(weeks=12))

In [4]:
# UK Population estimates (mid-year) by age
url = 'https://www.ons.gov.uk/visualisations/dvc845/poppyramids/pyramids/datadownload.xlsx'
response = cached_sess.get(url)
data = pd.ExcelFile(response.content)
pop = data.parse('2019', skiprows=[0])


In [5]:
pop

,variable,geogcode,geogname,m_19_0,m_19_1,m_19_2,m_19_3,m_19_4,m_19_5,m_19_6,...,f_19_82,f_19_83,f_19_84,f_19_85,f_19_86,f_19_87,f_19_88,f_19_89,f_19_90,f_19_al
0,Hartlepool,E06000001,Hartlepool,515.0,532.0,515.0,583.0,533.0,577.0,600.0,...,326.0,255.0,271.0,227.0,220.0,197.0,178.0,136.0,575.0,47830.0
1,Middlesbrough,E06000002,Middlesbrough,954.0,962.0,1010.0,1003.0,1009.0,1034.0,999.0,...,382.0,361.0,334.0,288.0,286.0,219.0,218.0,186.0,662.0,71261.0
2,Redcar and Cleveland,E06000003,Redcar and Cleveland,670.0,692.0,762.0,725.0,754.0,820.0,866.0,...,525.0,442.0,405.0,362.0,305.0,295.0,262.0,236.0,898.0,70571.0
3,Stockton-on-Tees,E06000004,Stockton-on-Tees,1089.0,1093.0,1183.0,1228.0,1292.0,1302.0,1340.0,...,560.0,529.0,489.0,432.0,407.0,391.0,360.0,295.0,1004.0,99755.0
4,Darlington,E06000005,Darlington,582.0,585.0,542.0,632.0,629.0,675.0,689.0,...,361.0,343.0,327.0,253.0,263.0,226.0,190.0,170.0,719.0,54738.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,ENGLAND AND WALES,K04000001,ENGLAND AND WALES,333765.0,347033.0,358209.0,369596.0,369309.0,372325.0,380825.0,...,171612.0,159775.0,147538.0,132916.0,121951.0,113232.0,102766.0,90545.0,371864.0,30057331.0
426,GREAT BRITAIN,K03000001,GREAT BRITAIN,359903.0,374092.0,386254.0,398863.0,398782.0,402075.0,411028.0,...,188277.0,175330.0,161761.0,145781.0,133356.0,123849.0,111979.0,98392.0,402111.0,32857628.0
427,UNITED KINGDOM,K02000001,UNITED KINGDOM,371576.0,386072.0,398693.0,411883.0,411576.0,414890.0,423939.0,...,193241.0,179876.0,165978.0,149501.0,136897.0,126965.0,114758.0,100820.0,411765.0,33818578.0
428,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Fixing the population dataframe
I can clear out the rows which aren't data at the bottom, and then combine the header columns as we're not interested in sex, only national populations and age groups
### Header format
The header format follows a pattern like `f_19_64` which accounts for females aged 64 at the mid-year 2019 estimate. As such, `sex (char(1)) underscore year of measure (char(2)) underscore age (char(2))`. As age 90 is greater than age 89, I assume age 90 includes all ages 90 and above.

In [6]:
# From above, we remove the last two rows as this does not contain data
pop = pop[:-2]


In [7]:
# Loop for all the age groups 0 through 90, create total column, drop male/female columns
for age in range(0,91):
    pop['t_19_'+str(age)] = pop['f_19_'+str(age)] + pop['m_19_'+str(age)]
    pop.drop(labels=['f_19_'+str(age), 'm_19_'+str(age)], axis=1, inplace=True)



<ipython-input-7-58aef569a439>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop['t_19_'+str(age)] = pop['f_19_'+str(age)] + pop['m_19_'+str(age)]
/usr/local/lib/python3.9/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [8]:
pop.head()

,variable,geogcode,geogname,m_19_al,f_19_al,t_19_0,t_19_1,t_19_2,t_19_3,t_19_4,...,t_19_81,t_19_82,t_19_83,t_19_84,t_19_85,t_19_86,t_19_87,t_19_88,t_19_89,t_19_90
0,Hartlepool,E06000001,Hartlepool,45833.0,47830.0,1006.0,1009.0,1031.0,1125.0,1058.0,...,598.0,544.0,461.0,462.0,384.0,373.0,317.0,252.0,230.0,830.0
1,Middlesbrough,E06000002,Middlesbrough,69719.0,71261.0,1802.0,1944.0,1979.0,1886.0,1972.0,...,769.0,675.0,627.0,574.0,484.0,476.0,347.0,374.0,279.0,925.0
2,Redcar and Cleveland,E06000003,Redcar and Cleveland,66579.0,70571.0,1313.0,1372.0,1508.0,1421.0,1492.0,...,956.0,919.0,787.0,705.0,603.0,531.0,487.0,423.0,393.0,1321.0
3,Stockton-on-Tees,E06000004,Stockton-on-Tees,97593.0,99755.0,2149.0,2131.0,2337.0,2366.0,2479.0,...,1093.0,992.0,918.0,864.0,692.0,681.0,664.0,569.0,463.0,1491.0
4,Darlington,E06000005,Darlington,52065.0,54738.0,1100.0,1152.0,1112.0,1218.0,1208.0,...,698.0,625.0,588.0,572.0,449.0,447.0,379.0,304.0,263.0,1064.0


## Definitions
### Adult
The definition of adult may vary according to vaccine. So we'll stick with the highest common value. This is 18 or older.

#### Pfizer/BioNTech 
Not recommended for those younger than 16. [source](https://www.gov.uk/government/publications/regulatory-approval-of-pfizer-biontech-vaccine-for-covid-19/information-for-uk-recipients-on-pfizerbiontech-covid-19-vaccine)

#### Astrazeneca
Efficacy and safety not tested on those younger than 18. [source](https://www.gov.uk/government/publications/regulatory-approval-of-covid-19-vaccine-astrazeneca/information-for-healthcare-professionals-on-covid-19-vaccine-astrazeneca)

#### Moderna
Efficacy and safety not established on those younger than 18. [source](https://www.gov.uk/government/publications/regulatory-approval-of-covid-19-vaccine-moderna/information-for-healthcare-professionals-on-covid-19-vaccine-moderna)

### Minor
Not an adult. This is 17 or younger.


In [9]:
# Use list comphrenesions to generate the column names we're going to be using
minors_colnms = ['t_19_'+str(age) for age in range(0,18)]
adults_colnms = ['t_19_'+str(age) for age in range(18,91)]

In [10]:
# Aggregate on both these columns, convert to integers
pop['t_19_minors'] = pop[minors_colnms].sum(axis=1).astype(int)
pop['t_19_adults'] = pop[adults_colnms].sum(axis=1).astype(int)

<ipython-input-10-3e1aa85d9d82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop['t_19_minors'] = pop[minors_colnms].sum(axis=1).astype(int)
<ipython-input-10-3e1aa85d9d82>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop['t_19_adults'] = pop[adults_colnms].sum(axis=1).astype(int)


# Vaccinations
Using the data from the Coronavirus dashboard at https://coronavirus.data.gov.uk, which is licensed under the [Open Government License v3.0](https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/).

In [11]:
rename = {'newPeopleVaccinatedSecondDoseByPublishDate': 'newSecondDose',
          'newPeopleVaccinatedFirstDoseByPublishDate': 'newFirstDose',
          'cumPeopleVaccinatedFirstDoseByPublishDate': 'cumFirstDose',
          'cumPeopleVaccinatedSecondDoseByPublishDate': 'cumSecondDose',
          'weeklyPeopleVaccinatedFirstDoseByVaccinationDate': 'newFirstDose',
          'weeklyPeopleVaccinatedSecondDoseByVaccinationDate': 'newSecondDose',
          'cumPeopleVaccinatedFirstDoseByVaccinationDate': 'cumFirstDose',
          'cumPeopleVaccinatedSecondDoseByVaccinationDate': 'cumSecondDose',
          't_19_adults': 'popMid2019adults'}

## For weekly vaccinations by nation

In [12]:
# Coronavirus dashboard vaccination data
# https://coronavirus.data.gov.uk/details/healthcare#card-people_who_have_received_vaccinations_by_nation
json = requests.get('https://coronavirus.data.gov.uk/api/v1/data?filters=areaType=nation&structure=%7B%22areaType%22:%22areaType%22,%22areaName%22:%22areaName%22,%22areaCode%22:%22areaCode%22,%22date%22:%22date%22,%22weeklyPeopleVaccinatedFirstDoseByVaccinationDate%22:%22weeklyPeopleVaccinatedFirstDoseByVaccinationDate%22,%22cumPeopleVaccinatedFirstDoseByVaccinationDate%22:%22cumPeopleVaccinatedFirstDoseByVaccinationDate%22,%22weeklyPeopleVaccinatedSecondDoseByVaccinationDate%22:%22weeklyPeopleVaccinatedSecondDoseByVaccinationDate%22,%22cumPeopleVaccinatedSecondDoseByVaccinationDate%22:%22cumPeopleVaccinatedSecondDoseByVaccinationDate%22%7D&format=json').json()
weekvac = pd.DataFrame(json['data'])
weekvac['date'] = pd.to_datetime(weekvac['date'], format='%Y-%m-%d') # Format date column
weekvac.head()

,areaType,areaName,areaCode,date,weeklyPeopleVaccinatedFirstDoseByVaccinationDate,cumPeopleVaccinatedFirstDoseByVaccinationDate,weeklyPeopleVaccinatedSecondDoseByVaccinationDate,cumPeopleVaccinatedSecondDoseByVaccinationDate
0,nation,England,E92000001,2021-02-28,2219150,17179491,85034,598345
1,nation,England,E92000001,2021-02-21,2050373,14960341,22559,513311
2,nation,England,E92000001,2021-02-14,2309588,12909968,18170,490752
3,nation,England,E92000001,2021-02-07,2406601,10600380,9319,472582
4,nation,England,E92000001,2021-01-31,2204262,8193779,8615,463263


In [13]:
# Bring in population data and drop duplicate column
weekdf = weekvac.merge(pop[['t_19_adults', 'geogcode']], how='inner', left_on='areaCode', right_on='geogcode')
weekdf.drop(['geogcode','areaType'], axis=1, inplace=True)
# Rename population value
weekdf = weekdf.rename(rename, axis=1)
# Labelling actuals
weekdf['Marker'], weekdf['Period'] = 'Actual', 'Week'
weekdf.head()

,areaName,areaCode,date,newFirstDose,cumFirstDose,newSecondDose,cumSecondDose,popMid2019adults,Marker,Period
0,England,E92000001,2021-02-28,2219150,17179491,85034,598345,44263393,Actual,Week
1,England,E92000001,2021-02-21,2050373,14960341,22559,513311,44263393,Actual,Week
2,England,E92000001,2021-02-14,2309588,12909968,18170,490752,44263393,Actual,Week
3,England,E92000001,2021-02-07,2406601,10600380,9319,472582,44263393,Actual,Week
4,England,E92000001,2021-01-31,2204262,8193779,8615,463263,44263393,Actual,Week


## For daily vaccinations by nation
What we're going to do is to download it, crop out anything which takes place before the latest weekly information, and append it to the dataframe, tweak the area charts' time axis and labels, to show more detail.


In [14]:
# Coronavirus daily dashboard vaccination data
# https://coronavirus.data.gov.uk/details/healthcare#card-people_who_have_received_vaccinations_by_nation
json = requests.get('https://coronavirus.data.gov.uk/api/v1/data?filters=areaType=nation&structure=%7B%22areaType%22:%22areaType%22,%22areaName%22:%22areaName%22,%22areaCode%22:%22areaCode%22,%22date%22:%22date%22,%22newPeopleVaccinatedFirstDoseByPublishDate%22:%22newPeopleVaccinatedFirstDoseByPublishDate%22,%22newPeopleVaccinatedSecondDoseByPublishDate%22:%22newPeopleVaccinatedSecondDoseByPublishDate%22,%22cumPeopleVaccinatedFirstDoseByPublishDate%22:%22cumPeopleVaccinatedFirstDoseByPublishDate%22,%22cumPeopleVaccinatedSecondDoseByPublishDate%22:%22cumPeopleVaccinatedSecondDoseByPublishDate%22%7D&format=json').json()
dayvac = pd.DataFrame(json['data'])
dayvac['date'] = pd.to_datetime(dayvac['date'], format='%Y-%m-%d') # Format date column
dayvac.head()


,areaType,areaName,areaCode,date,newPeopleVaccinatedFirstDoseByPublishDate,newPeopleVaccinatedSecondDoseByPublishDate,cumPeopleVaccinatedFirstDoseByPublishDate,cumPeopleVaccinatedSecondDoseByPublishDate
0,nation,England,E92000001,2021-03-12,312716.0,67514.0,20111189,1076426
1,nation,England,E92000001,2021-03-11,211393.0,69680.0,19798473,1008912
2,nation,England,E92000001,2021-03-10,206720.0,70960.0,19587080,939232
3,nation,England,E92000001,2021-03-09,181127.0,47151.0,19380360,868272
4,nation,England,E92000001,2021-03-08,183736.0,23800.0,19199233,821121


In [15]:
# Bring in population data and drop duplicate column
daydf = dayvac.merge(pop[['t_19_adults', 'geogcode']], how='inner', left_on='areaCode', right_on='geogcode')
daydf.drop(['geogcode','areaType'], axis=1, inplace=True)
daydf = daydf.rename(rename, axis=1)
# Labelling actuals
daydf['Marker'], daydf['Period'] = 'Actual', 'Day'
daydf.head()

,areaName,areaCode,date,newFirstDose,newSecondDose,cumFirstDose,cumSecondDose,popMid2019adults,Marker,Period
0,England,E92000001,2021-03-12,312716.0,67514.0,20111189,1076426,44263393,Actual,Day
1,England,E92000001,2021-03-11,211393.0,69680.0,19798473,1008912,44263393,Actual,Day
2,England,E92000001,2021-03-10,206720.0,70960.0,19587080,939232,44263393,Actual,Day
3,England,E92000001,2021-03-09,181127.0,47151.0,19380360,868272,44263393,Actual,Day
4,England,E92000001,2021-03-08,183736.0,23800.0,19199233,821121,44263393,Actual,Day


In [16]:
daydf.columns, weekdf.columns

(Index(['areaName', 'areaCode', 'date', 'newFirstDose', 'newSecondDose',
        'cumFirstDose', 'cumSecondDose', 'popMid2019adults', 'Marker',
        'Period'],
       dtype='object'),
 Index(['areaName', 'areaCode', 'date', 'newFirstDose', 'cumFirstDose',
        'newSecondDose', 'cumSecondDose', 'popMid2019adults', 'Marker',
        'Period'],
       dtype='object'))

In [17]:
combineddf =pd.concat([daydf, weekdf])

In [18]:
combineddf.head()

,areaName,areaCode,date,newFirstDose,newSecondDose,cumFirstDose,cumSecondDose,popMid2019adults,Marker,Period
0,England,E92000001,2021-03-12,312716.0,67514.0,20111189,1076426,44263393,Actual,Day
1,England,E92000001,2021-03-11,211393.0,69680.0,19798473,1008912,44263393,Actual,Day
2,England,E92000001,2021-03-10,206720.0,70960.0,19587080,939232,44263393,Actual,Day
3,England,E92000001,2021-03-09,181127.0,47151.0,19380360,868272,44263393,Actual,Day
4,England,E92000001,2021-03-08,183736.0,23800.0,19199233,821121,44263393,Actual,Day


# Model

In [19]:
# Calculate the daynumbers Monday = 0, ..., Sunday = 6
combineddf['day'] = combineddf["date"].apply(lambda x: x.weekday())

In [20]:
combineddf['newDose'] = combineddf['newFirstDose'] + combineddf['newSecondDose']

In [21]:
combineddf

,areaName,areaCode,date,newFirstDose,newSecondDose,cumFirstDose,cumSecondDose,popMid2019adults,Marker,Period,day,newDose
0,England,E92000001,2021-03-12,312716.0,67514.0,20111189,1076426,44263393,Actual,Day,4,380230.0
1,England,E92000001,2021-03-11,211393.0,69680.0,19798473,1008912,44263393,Actual,Day,3,281073.0
2,England,E92000001,2021-03-10,206720.0,70960.0,19587080,939232,44263393,Actual,Day,2,277680.0
3,England,E92000001,2021-03-09,181127.0,47151.0,19380360,868272,44263393,Actual,Day,1,228278.0
4,England,E92000001,2021-03-08,183736.0,23800.0,19199233,821121,44263393,Actual,Day,0,207536.0
...,...,...,...,...,...,...,...,...,...,...,...,...
43,Wales,W92000004,2021-01-10,41726.0,83.0,92482,110,2522940,Actual,Week,6,41809.0
44,Wales,W92000004,2021-01-03,14205.0,27.0,50756,27,2522940,Actual,Week,6,14232.0
45,Wales,W92000004,2020-12-27,12495.0,0.0,36551,0,2522940,Actual,Week,6,12495.0
46,Wales,W92000004,2020-12-20,15799.0,0.0,24056,0,2522940,Actual,Week,6,15799.0


In [44]:
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.pyplot as plt

In [24]:
df = combineddf.copy()

In [25]:
df.head()

,areaName,areaCode,date,newFirstDose,newSecondDose,cumFirstDose,cumSecondDose,popMid2019adults,Marker,Period,day,newDose
0,England,E92000001,2021-03-12,312716.0,67514.0,20111189,1076426,44263393,Actual,Day,4,380230.0
1,England,E92000001,2021-03-11,211393.0,69680.0,19798473,1008912,44263393,Actual,Day,3,281073.0
2,England,E92000001,2021-03-10,206720.0,70960.0,19587080,939232,44263393,Actual,Day,2,277680.0
3,England,E92000001,2021-03-09,181127.0,47151.0,19380360,868272,44263393,Actual,Day,1,228278.0
4,England,E92000001,2021-03-08,183736.0,23800.0,19199233,821121,44263393,Actual,Day,0,207536.0


In [93]:
# For England
england = df.loc[(df['areaName'] == 'England') & (df['Period'] == 'Day'), ['date', 'newDose']].set_index('date').dropna()
eng_result = seasonal_decompose(np.array(england), model='multiplactive', period=7)

In [94]:
# For Scotland
scotland = df.loc[(df['areaName'] == 'Scotland') & (df['Period'] == 'Day'), ['date', 'newDose']].set_index('date').dropna()
sco_result = seasonal_decompose(np.array(scotland), model='multiplactive', period=7)

In [95]:
# For Wales
wales = df.loc[(df['areaName'] == 'Wales') & (df['Period'] == 'Day'), ['date', 'newDose']].set_index('date').dropna()
wal_result = seasonal_decompose(np.array(scotland), model='multiplactive', period=7)

In [98]:
# For Northern Ireland
nie = df.loc[(df['areaName'] == 'Northern Ireland') & (df['Period'] == 'Day'), ['date', 'newDose']].set_index('date').dropna()
nie_result = seasonal_decompose(np.array(scotland), model='multiplactive', period=7)

In [127]:
# England seasonality 
seasonality = pd.DataFrame([eng_result.seasonal[0:7], sco_result.seasonal[0:7], wal_result.seasonal[0:7], nie_result.seasonal[0:7], (1, 2, 3, 4, 5, 6, 0)]).T.set_index(4)
seasonality.rename({0: 'England', 1: 'Scotland', 2: 'Wales', 3: 'N. Ireland'}, axis=1, inplace=True)
seasonality.index = seasonality.index.astype(int)
seasonality.index.name = 'Weekday'
seasonality.sort_index(inplace=True)
seasonality.to_csv('test.csv')

In [110]:
plt.close('all')
#define chart parameters
plt.style.use('seaborn')


fig = plt.figure(figsize=(8,6))

gs = gridspec.GridSpec(2,2, width_ratios=[4,2])

a0 = plt.subplot(gs[0])
a1 = plt.subplot(gs[1])
a2 = plt.subplot(gs[2])
a3 = plt.subplot(gs[3])

eng_result.plot()

fig_title = f"United Kingdom's Vaccination Progress to {shape_first_day.index.max().strftime('%a %d %b %Y')}"
fig.suptitle(fig_title, fontsize=15)

plt.show()
fig.savefig(f"decomp_model_multiplicative_{shape_first_day.index.max().strftime('%Y%m%d')}")

TypeError: float() argument must be a string or a number, not 'DecomposeResult'

In [123]:
eng_result.resid

array([       nan,        nan,        nan, 0.93934603, 1.09216765,
       0.86135151, 1.07172281, 1.11775202, 1.06681439, 0.89467773,
       0.82808988, 0.87707447, 0.95147746, 0.92630125, 1.07126006,
       1.11797871, 1.1826793 , 1.05872676, 0.80550726, 0.6837045 ,
       0.82320512, 0.85670547, 1.10504495, 1.28290927, 1.08404364,
       0.94812138, 0.92384391, 1.00518367, 1.01680644, 0.96738217,
       0.93832888, 1.02478508, 1.09965236, 1.13947615, 1.02475628,
       0.89680175, 0.93264511, 0.99846776, 0.92030871, 1.11077492,
       1.33904565, 1.21348128, 0.97298663, 0.84725163, 0.7295435 ,
       0.95010465, 1.13324077, 1.08537231, 1.07370201, 1.00050543,
       0.94574734, 0.96877959, 1.18932215, 0.92818808, 1.01045541,
       0.85637448, 1.06190908, 1.01186258,        nan,        nan,
              nan])

In [124]:
df.columns

Index(['areaName', 'areaCode', 'date', 'newFirstDose', 'newSecondDose',
       'cumFirstDose', 'cumSecondDose', 'popMid2019adults', 'Marker', 'Period',
       'day', 'newDose'],
      dtype='object')

In [125]:
df['date'].max

<bound method NDFrame._add_numeric_operations.<locals>.max of 0    2021-03-12
1    2021-03-11
2    2021-03-10
3    2021-03-09
4    2021-03-08
        ...    
43   2021-01-10
44   2021-01-03
45   2020-12-27
46   2020-12-20
47   2020-12-13
Name: date, Length: 296, dtype: datetime64[ns]>

In [126]:
df[(df['date'] == df['date'].max())]

,areaName,areaCode,date,newFirstDose,newSecondDose,cumFirstDose,cumSecondDose,popMid2019adults,Marker,Period,day,newDose
0,England,E92000001,2021-03-12,312716.0,67514.0,20111189,1076426,44263393,Actual,Day,4,380230.0
62,Northern Ireland,N92000002,2021-03-12,6833.0,652.0,621462,50052,1452962,Actual,Day,4,7485.0
124,Scotland,S92000003,2021-03-12,22487.0,6841.0,1867123,156250,4434138,Actual,Day,4,29328.0
186,Wales,W92000004,2021-03-12,27542.0,12669.0,1084329,250026,2522940,Actual,Day,4,40211.0
